In [1]:
# imports
import requests
import pandas as pd
import os

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [2]:
# Load bike station data from previously exported .csv (See citybikes.ipynb)
station_data = pd.read_csv("../data/citybike_stations.csv")

station_data.head(5)

,station_id,station_name,free_bikes,empty_slots,latitude,longitude
0,af1d0f25cbc75377878349fde4d86133,Merchant Square - ELECTRIC,7,2.0,55.858167,-4.245483
1,71b02b6f26f03e58c97dec1e1c9faa00,Botanic Gardens - ELECTRIC,1,7.0,55.878278,-4.288487
2,fa6bc682e8627d66891c2b6250c81b1e,Mitchell Library,7,0.0,55.864788,-4.271852
3,a087989280fd941fd9c41c3e787940ae,University of Strathclyde (North),1,2.0,55.862983,-4.241000
4,eb47a36e3d5fe5bc456397413bd2aa75,Bridge Street Subway - ELECTRIC,5,3.0,55.852500,-4.258850


In [3]:
# Create a dictionary containing the station id (for later use), and formatted coordinates to be used in the below API call
station_coordinates = []

# For every item & row in the CSV
for index, row in station_data.iterrows():
    # Gather the id, lat & long for each station
    station_id = row['station_id']
    lat = row['latitude']
    lon = row['longitude']
    # Load the values into a dictionary
    station_data = {'station_id': station_id, 'coords': f"{lat},{lon}"}
    # Add it to our coordinate list
    station_coordinates.append(station_data)

# Headers & parameters needed for FourSquare
headers = {"Accept": "application/json"}
headers['Authorization'] = os.environ["FOURSQUARE_API_KEY"]
radius_to_search = 1000 # Distance of POI from bike station location (in meters)

# Send an API request for every pair of station coordinates to compile POI data
poi_data = []
for station in station_coordinates:
    # Create the API call using the bike station location and provided search radius
    request_url = "https://api.foursquare.com/v3/places/search?ll=" + station['coords'] + "&radius=" + str(radius_to_search)
    response = requests.get(request_url, headers=headers)
    poi_data.append({'station_id': station['station_id'], 'api-data': response.json()}) # Return the station ID and JSON response

Load the API data into a DataFrame

In [4]:
# NOTE to self: Run this code block to avoid repeated API calls

# Load the compiled data into Pandas dataframe 
poi_data_df = pd.DataFrame(poi_data)

poi_data_df.head(5)

,station_id,api-data
0,af1d0f25cbc75377878349fde4d86133,{'results': [{'fsq_id': '54c7cade498e300defb09...
1,71b02b6f26f03e58c97dec1e1c9faa00,{'results': [{'fsq_id': '4b9bc057f964a5201c213...
2,fa6bc682e8627d66891c2b6250c81b1e,{'results': [{'fsq_id': '4b852d7bf964a5207d4f3...
3,a087989280fd941fd9c41c3e787940ae,{'results': [{'fsq_id': '57c45369498e5276e3a65...
4,eb47a36e3d5fe5bc456397413bd2aa75,{'results': [{'fsq_id': '4b1583cef964a520fbad2...


In [5]:
# This cell will extract the API data using a loop, creating a new data frame with the station_id and contents of api-data

results_df_list = [] # Empty list to store our results

# Iterate through every row in the POI dataframe
for index, row in poi_data_df.iterrows():
    # Extract the foursquare results from the data column
    results = row['api-data']['results']
    
    # Check if the data exists (without this, excess blank rows will be created)
    if len(results) > 0:
        # Populate a dataframe with the extracted data
        results_df = pd.DataFrame(results)
        # Extract the station ID from the Citybikes API
        results_df['station_id'] = row['station_id']

        results_df_list.append(results_df)

# Combine the results from this list across the rows (axis 0) back into our dataframe
poi_data_df = pd.concat(results_df_list, axis=0)

poi_data_df.head(5)

,fsq_id,categories,chains,distance,geocodes,link,location,name,related_places,timezone,station_id
0,54c7cade498e300defb0985f,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",[],31,"{'main': {'latitude': 55.858485, 'longitude': ...",/v3/places/54c7cade498e300defb0985f,"{'address': '6 Wilson St', 'admin_region': 'Sc...",Wilson Street Pantry,{},Europe/London,af1d0f25cbc75377878349fde4d86133
1,4bc35be22a89ef3b08e7f488,"[{'id': 13199, 'name': 'Indian Restaurant', 'i...",[],34,"{'main': {'latitude': 55.858078, 'longitude': ...",/v3/places/4bc35be22a89ef3b08e7f488,"{'address': '44 Candleriggs', 'admin_region': ...",The Dhabba,{},Europe/London,af1d0f25cbc75377878349fde4d86133
2,4b9cecbcf964a520bd8236e3,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...",[],43,"{'main': {'latitude': 55.858429, 'longitude': ...",/v3/places/4b9cecbcf964a520bd8236e3,"{'address': '71-73 Albion St', 'admin_region':...",Merchant Square,{'children': [{'fsq_id': '595f841bf79faa623f93...,Europe/London,af1d0f25cbc75377878349fde4d86133
3,4ba0e8c3f964a520508637e3,"[{'id': 10037, 'name': 'Concert Hall', 'icon':...",[],98,"{'main': {'latitude': 55.85886, 'longitude': -...",/v3/places/4ba0e8c3f964a520508637e3,"{'admin_region': 'Scotland', 'country': 'GB', ...",City Halls,{},Europe/London,af1d0f25cbc75377878349fde4d86133
4,554cdd3c498ee17d5a870117,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...",[],144,"{'main': {'latitude': 55.859412, 'longitude': ...",/v3/places/554cdd3c498ee17d5a870117,"{'address': '127 Candleriggs', 'admin_region':...",Spitfire Espresso,{},Europe/London,af1d0f25cbc75377878349fde4d86133


In [6]:
# Renaming some columns to make the contents more obvious
poi_data_df.rename(columns={'distance':'distance_from_station'}, inplace=True)
poi_data_df.rename(columns={'name':'poi_name'}, inplace=True)

# Display altered column headers
poi_data_df.head(1)

,fsq_id,categories,chains,distance_from_station,geocodes,link,location,poi_name,related_places,timezone,station_id
0,54c7cade498e300defb0985f,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",[],31,"{'main': {'latitude': 55.858485, 'longitude': ...",/v3/places/54c7cade498e300defb0985f,"{'address': '6 Wilson St', 'admin_region': 'Sc...",Wilson Street Pantry,{},Europe/London,af1d0f25cbc75377878349fde4d86133


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [7]:
# Copy the desired columns into the final dataframe
selected_poi_data_df = poi_data_df[['station_id','distance_from_station','poi_name','categories','location']].copy()

# Display results
selected_poi_data_df.head(5)

,station_id,distance_from_station,poi_name,categories,location
0,af1d0f25cbc75377878349fde4d86133,31,Wilson Street Pantry,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...","{'address': '6 Wilson St', 'admin_region': 'Sc..."
1,af1d0f25cbc75377878349fde4d86133,34,The Dhabba,"[{'id': 13199, 'name': 'Indian Restaurant', 'i...","{'address': '44 Candleriggs', 'admin_region': ..."
2,af1d0f25cbc75377878349fde4d86133,43,Merchant Square,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...","{'address': '71-73 Albion St', 'admin_region':..."
3,af1d0f25cbc75377878349fde4d86133,98,City Halls,"[{'id': 10037, 'name': 'Concert Hall', 'icon':...","{'admin_region': 'Scotland', 'country': 'GB', ..."
4,af1d0f25cbc75377878349fde4d86133,144,Spitfire Espresso,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...","{'address': '127 Candleriggs', 'admin_region':..."


In [8]:
# This cell creates and uses two functions to extract the category names & address into seperate columns for legibility

# Function used in conjunction with .apply() to extract category "names" into a easier to read list
def get_category_names(row):
    # Load the dictionary from the categories column
    categories_dic = row['categories']
    
    # Extract the names of each category into a tuple
    # NOTE: Tried list before, but .drop_duplicates() does not work on that data type so I had to settle with tuples
    category_tuple = ()
    for category in categories_dic:
        category_tuple += (category['name'],) # Append the name to the category tuple
        
    # Return the tuple of category name
    return category_tuple

# Function used in conjunction with .apply() to extract the address from a location, as we do not require the other data
def get_address(row):
    # Load the dictionary from the location row
    location_dic = row['location']
    
    # Extract the address from the location dictionary
    poi_address = location_dic.get('address', '')
    
    return poi_address

# Use pandas .apply() dataframe function to extract the important values into new columns for much easier analysis
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html
selected_poi_data_df['category_names'] = selected_poi_data_df.apply(get_category_names, axis=1)
selected_poi_data_df['address'] = selected_poi_data_df.apply(get_address, axis=1)

# Display the resulting DataFrame
selected_poi_data_df.head(5)

,station_id,distance_from_station,poi_name,categories,location,category_names,address
0,af1d0f25cbc75377878349fde4d86133,31,Wilson Street Pantry,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...","{'address': '6 Wilson St', 'admin_region': 'Sc...","(Café, Restaurant)",6 Wilson St
1,af1d0f25cbc75377878349fde4d86133,34,The Dhabba,"[{'id': 13199, 'name': 'Indian Restaurant', 'i...","{'address': '44 Candleriggs', 'admin_region': ...","(Indian Restaurant,)",44 Candleriggs
2,af1d0f25cbc75377878349fde4d86133,43,Merchant Square,"[{'id': 13003, 'name': 'Bar', 'icon': {'prefix...","{'address': '71-73 Albion St', 'admin_region':...","(Bar, Restaurant)",71-73 Albion St
3,af1d0f25cbc75377878349fde4d86133,98,City Halls,"[{'id': 10037, 'name': 'Concert Hall', 'icon':...","{'admin_region': 'Scotland', 'country': 'GB', ...","(Concert Hall, Theater)",
4,af1d0f25cbc75377878349fde4d86133,144,Spitfire Espresso,"[{'id': 13035, 'name': 'Coffee Shop', 'icon': ...","{'address': '127 Candleriggs', 'admin_region':...","(Coffee Shop, Restaurant)",127 Candleriggs


Drop the columns that are no longer needed & remove duplicates

In [ ]:
# Drop columns
selected_poi_data_df = selected_poi_data_df.drop(['categories','location'], axis=1) # - Commented out as I've already run before and keep getting 'doesn't exist' errors

# Remove duplicates (Will happen in cases when two bike stations are within the search radius of eachother)
# NOTE: Stopped working once I added in the station_id as the rows were no longer exact dupes, had to make function below
#selected_poi_data_df = selected_poi_data_df.drop_duplicates()

# Remove rows with duplicate names & addresses, keeping the value with the shortest distance
# ie. Only keep the shortest path per one venue, otherwise analysis will be inaccurate
shortest_distance_rows = {}
# For every row in the dataframe
for index, row in selected_poi_data_df.iterrows():
    # Gather the important data from current row
    poi_name = row['poi_name']
    address = row['address']
    distance_from_station = row['distance_from_station']
    
    # Check if we have already seen this combination before (ie duplicate POI)
    if (poi_name, address) in shortest_distance_rows:
        # Is the distance to station shorter than what we have saved?
        if distance_from_station < shortest_distance_rows[(poi_name, address)]['distance_from_station']:
            # If so, update our dictionary
            shortest_distance_rows[(poi_name, address)] = row
    else:
        # Otherwise, add it to our dictionary (unseen POI)
        shortest_distance_rows[(poi_name, address)] = row

# Load the compiled values into our dataframe
selected_poi_data_df = pd.DataFrame(shortest_distance_rows.values())

# View the final dataframe to be exported to .CSV
selected_poi_data_df

In [ ]:
# Export to .csv
selected_poi_data_df.to_csv('../data/Foursquare_POIs.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [ ]:
# Load Citybik.es export (See citybikes.ipynb)
station_data = pd.read_csv("../data/citybike_stations.csv")

station_data.head(5)

In [ ]:
# Headers & parameters needed for Yelp API acording to docs
# https://docs.developer.yelp.com/docs/fusion-intro
yelp_headers = {"accept": "application/json"}
yelp_headers['Authorization'] = "Bearer " + os.environ["YELP_API_KEY"]
radius_to_search = 1000

In [ ]:
# YELP API LOOP
# -- Brought into a seperate cell to restrict the amount of times it needs to be called, reducing API usage
yelp_POIs = []          # List to store POI results
#api_call_limit = 3      # Limit the amount of times the API loop will run (for testing) 
api_limit_counter = 0   # ^

# For each station in the loaded CSV
for index, row in station_data.iterrows():
    #if api_limit_counter >= api_call_limit: # Number of times I want to limit the API to run, used for testing
    #    break
    
    # Gather the id, lat & long for each station
    station_id = row['station_id']
    lat = row['latitude']
    lon = row['longitude']
    
    # Form the API request URL
    yelp_request_url = "https://api.yelp.com/v3/businesses/search?latitude=" + str(lat) + "&longitude=" + str(lon) + "&radius=" + str(radius_to_search)
    
    # Send & recieve the response
    response = requests.get(yelp_request_url, headers=yelp_headers)
    
    # Load the 
    yelp_POIs.append({'station_id': station['station_id'], 'api-data': response.json()['businesses']})
    
    api_limit_counter += 1

Put your parsed results into a DataFrame

In [ ]:
# Load the station ID & API response into a dataframe
# NOTE: Use this cell to reset the DF instead of above to avoid unnessasary API calls
yelp_POIs_df = pd.DataFrame(yelp_POIs)
yelp_POIs_df.head(5)

In [ ]:
# Now similarly to the Foursquare API we will have to extract the results for each station ID into their own rows

yelp_results_df_list = []

# Iterate through every row in the POI dataframe
for index, row in yelp_POIs_df.iterrows():
    # Extract the foursquare results from the data column
    results = row['api-data']
    
    # Check if the data exists (without this, excess blank rows will be created)
    if len(results) > 0:
        # Populate a dataframe with the extracted data
        results_df = pd.DataFrame(results)
        # Extract the station ID from the Citybikes API
        results_df['station_id'] = row['station_id']

        yelp_results_df_list.append(results_df)

# Combine the results from this list across the rows (axis 0) back into our dataframe
yelp_POIs_df = pd.concat(yelp_results_df_list, axis=0)

In [ ]:
yelp_POIs_df.head(5)

In [ ]:
# Rename a few columns to make them less ambiguous
yelp_POIs_df.rename(columns={'name':'POI_name','distance':'distance_from_station'}, inplace=True)

yelp_POIs_df.head(1)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [ ]:
# Copy over the columns to analyse
selected_yelp_POIs_df = yelp_POIs_df[['station_id','distance_from_station','POI_name','categories','location','rating','review_count','price']].copy()
selected_yelp_POIs_df.head(5)

In [ ]:
# Gather Addresss and Catagory names using the previously created functions used on the Foursquare data

# Function used in conjunction with .apply() to extract the address from a location, as we do not require the other data
def get_yelp_address(row):
    # Load the dictionary from the location row
    location_dic = row['location']
    
    # Extract the address from the location dictionary
    poi_address = location_dic.get('address1', '')
    
    return poi_address

# Function used in conjunction with .apply() to extract category "names" into a easier to read list
def get_yelp_category_names(row):
    # Load the dictionary from the categories column
    categories_dic = row['categories']
    
    # Extract the names of each category into a list
    category_names = []
    for category in categories_dic:
        category_names.append(category['title']) # Append the name to the category tuple
        
    # Return the tuple of category name
    return category_names


selected_yelp_POIs_df['address'] = selected_yelp_POIs_df.apply(get_yelp_address, axis=1)
selected_yelp_POIs_df['category_names'] = selected_yelp_POIs_df.apply(get_yelp_category_names, axis=1)

selected_yelp_POIs_df.head(5)

In [ ]:
print(f'Shape before de-duplication {selected_yelp_POIs_df.shape}')

In [ ]:
# Drop columns that we have already parsed
selected_yelp_POIs_df = selected_yelp_POIs_df.drop(['categories','location'], axis=1)

# Remove rows with duplicate names & addresses, keeping the value with the shortest distance
# ie. Only keep the shortest path per one venue, otherwise analysis will be inaccurate
shortest_distance_rows = {}
# For every row in the dataframe
for index, row in selected_yelp_POIs_df.iterrows():
    # Gather the important data from current row
    poi_name = row['POI_name']
    address = row['address']
    distance_from_station = row['distance_from_station']
    
    # Check if we have already seen this combination before (ie duplicate POI)
    if (poi_name, address) in shortest_distance_rows:
        # Is the distance to station shorter than what we have saved?
        if distance_from_station < shortest_distance_rows[(poi_name, address)]['distance_from_station']:
            # If so, update our dictionary
            shortest_distance_rows[(poi_name, address)] = row
    else:
        # Otherwise, add it to our dictionary (unseen POI)
        shortest_distance_rows[(poi_name, address)] = row

# Load the compiled values into our dataframe
selected_yelp_POIs_df = pd.DataFrame(shortest_distance_rows.values())

# Final view before export to .csv
selected_yelp_POIs_df.head(5)

In [ ]:
print(f'Shape after de-duplication {selected_yelp_POIs_df.shape}')

In [ ]:
# Export to .csv
selected_yelp_POIs_df.to_csv('../data/Yelp_POIs.csv', index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

In my opinion the Yelp API provided more complete data. Not only did Yelp have an extra ~100 points of interest, but the quality and amount of data was much better (16 columns vs 10).

Get the top 10 restaurants according to their rating

In [ ]:
# Top venues by rating (you'll notice half have a small sample size of reviews)
selected_yelp_POIs_df.sort_values('rating', ascending=False).head(10)

In [ ]:

# To get more accurate results, lets limit our search to places with 10 or more reviews
top_glasgow_restaurants = selected_yelp_POIs_df[selected_yelp_POIs_df['review_count'] >= 10]

# Sort by rating
top_glasgow_restaurants = top_glasgow_restaurants.sort_values('rating', ascending=False)

# Select only the columns we need for this analysis
top_glasgow_restaurants = top_glasgow_restaurants[['POI_name', 'category_names', 'rating', 'price', 'address']]

# Limit to top 10
top_glasgow_restaurants.head(10)

In [ ]:
# We can see that some non restaurant items were included, because Yelp does not have a "restaurant" 
# category so we will have to filter out unwanted values

values_to_remove = ["Yelp Events","Parks"]

# The following function looks through each tuple and will return true or false if the value is found or not
# if the value returns "false" it will be removed from the dataframe
def check_value(category_list):
    not_found = True
    
    # for every value we'd like to remove
    for value in values_to_remove:
        # if it's found
        if value in category_list:
            # return that it's been found (ie not not_found lol)
            not_found = False
            break
    
    return not_found

# Apply the filter
top_glasgow_restaurants_filtered = top_glasgow_restaurants[top_glasgow_restaurants['category_names'].apply(check_value)]

top_glasgow_restaurants_filtered.head(10)